<a href="https://colab.research.google.com/github/bhanuchaddha/Understanding-RAG/blob/main/BasicRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Basic RAG (Retrieval-Augmented Generation)**

### **Introduction:**
Retrieval-Augmented Generation (RAG) combines information retrieval and text generation by leveraging pre-trained language models and vector-based search. In this example, we are using:
- **FAISS** (Facebook AI Similarity Search) to index document embeddings and retrieve the most relevant ones.
- **GPT-Neo** for text generation based on the query and retrieved documents.
- **Gradio** to build a simple interactive UI to input a query and get a response.

### **Step 1: Install Required Libraries**
We need to install a few libraries to run the RAG model:
- **transformers**: to load pre-trained models like `bert-base-uncased` and `gpt-neo`.
- **faiss-cpu**: for efficient similarity search and document retrieval.
- **gradio**: to create the interactive user interface.
- **torch**: as it's needed to handle PyTorch models.

In [6]:
!pip install transformers faiss-cpu torch gradio langchain

### **Step 2: Import Necessary Libraries**
In this step, we import the libraries needed for building the RAG system:
- `AutoTokenizer` and `AutoModel` are used for handling BERT-based embeddings.
- `faiss` is used to create an index for document embeddings and handle similarity searches.
- `pipeline` helps load GPT-Neo for text generation.

In [7]:
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel, pipeline

### **Step 3: Loading Tokenizer and Model for Embeddings**
Here, we load a pre-trained BERT model (`bert-base-uncased`) and its tokenizer. This model will be used to generate embeddings for our documents.

In [8]:
# Load the tokenizer and model for embeddings
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

### **Step 4: Define Documents**
We define a list of documents that will be used to retrieve relevant information based on the user's query. These are simple sentences about AI and self-driving cars.

In [9]:
documents = [
    "AI is revolutionizing self-driving technology.",
    "Recent AI advancements focus on safety in autonomous vehicles.",
    "AI is helping to reduce human errors in self-driving cars.",
    "Large language models are driving advancements in machine learning."
]

### **Step 5: Function to Generate Embeddings**
This function, `get_embeddings`, takes in a list of text strings, tokenizes them, and generates vector embeddings using the BERT model. These embeddings are what we will index using FAISS for similarity search.

In [10]:
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

### **Step 6: Compute Embeddings for the Documents**
We use the `get_embeddings` function to convert the documents into embeddings. Each document is represented as a vector in this step, which can later be indexed for similarity searches.

In [11]:
# Compute embeddings for documents
embeddings = get_embeddings(documents)

### **Step 7: Indexing the Embeddings using FAISS**
FAISS is used to index the document embeddings. This allows us to efficiently search for documents that are most similar to the user's query. Here, we create a FAISS index and add the document embeddings to it.

In [12]:
# Index the embeddings using FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

### **Step 8: Load GPT-Neo for Text Generation**
We load GPT-Neo (specifically `EleutherAI/gpt-neo-1.3B`), a model capable of generating human-like text. This will be used to generate a response based on the query and relevant documents.

In [13]:
# Load GPT-Neo for text generation
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

### **Step 9: Retrieval and Generation Function**
The `retrieve_and_generate` function takes a user's query, generates its embedding, retrieves the top-k most relevant documents, and uses those documents to help GPT-Neo generate a response.
- First, it calculates the embedding of the query and finds the nearest documents in the FAISS index.
- Then, it combines the retrieved documents with the query to create a 'prompt' for GPT-Neo, which will generate the response.

In [14]:

def retrieve_and_generate(query, top_k=2):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, top_k)
    retrieved_docs = [documents[idx] for idx in indices[0]]
    prompt = query + "\n\nRelevant information:\n" + "\n".join(retrieved_docs)
    generated_text = generator(prompt, max_length=100, do_sample=False)[0]['generated_text']
    return generated_text


### **Step 10: Gradio Interface Setup**
We use **Gradio** to create a simple user interface where the user can input a query. The Gradio interface will call the `retrieve_and_generate` function to display the generated response.

In [15]:

import gradio as gr

interface = gr.Interface(
    fn=retrieve_and_generate,
    inputs='text',
    outputs='text',
    title='Retrieval-Augmented Generation (RAG) Demo',
    description='Ask a question related to AI and see RAG in action. It retrieves relevant information and generates a response using GPT-Neo.'
)

interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29b1f675571b06e272.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### **Step 11: Run the Gradio Interface**
Finally, we launch the Gradio app. This creates an interface where users can type a query and get a response generated by the RAG model.

After running the code, you’ll get a link where you can interact with the model through a simple web UI.

### **Example Query:**
You can test the app with the following query:

```plaintext
Tell me about the latest AI trends in self-driving cars.
```

The RAG model will retrieve the most relevant documents and use them to generate a response using GPT-Neo.

### **Conclusion:**
This demonstrates a basic Retrieval-Augmented Generation (RAG) model with embeddings, FAISS for similarity search, GPT-Neo for text generation, and Gradio for UI interaction. This setup provides a foundation for building more advanced systems with additional documents, more complex retrieval mechanisms, or fine-tuned models.